# **Assignment 4: Question Answering**
**Due**: Monday, January 29, 2024, 2pm, via [Moodle](https://moodle.uni-heidelberg.de/course/view.php?id=19251)



### **Submission Guidelines**

- Solutions need to be uploaded as a **single** Jupyter notebook. You will find several pre-filled code segments in the notebook, your task is to fill in the missing cells.
- For the written solution, use LaTeX in markdown inside the same notebook. Do **not** hand in a separate file for it.
- Download the .zip file containing the dataset but do **not** upload it with your solution.
- It is sufficient if one person per group uploads the solution to Moodle, but make sure that the full names of all team members are given in the notebook.

***

## **Task 1: Retrieval Augmented Generation (RAG)** ( 4.5 + 3 + 4 + 3 + 1.5 = 16 points)

In this task, we look at using the open source `Llama-13b-chat` model for creating a RAG system. You must first apply for access to Llama 2 models via [this](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) form (access is typically granted within a few hours). etrieval augmented generation you also need to request to use the model on Hugging Face by going to the [model](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) card. ***Note that the emails you provide for your Hugging Face account must match the email you used to request Llama 2.***

The final piece that you need is a Hugging Face authentication token. You can find such a token by going to the `setting` in your Hugging Face profile, under the `Access Token` menu you can generate a new token.

To store the document you will need a free Pinecone [API key](https://app.pinecone.io/).
Make sure you have these pieces ready before starting to work on this task.

----
When ready, let's start by downloading the necessary packages.

It is advised to proceed with this notebook with a GPU (if you are on Colab make sure that a GPU environment is activated.)


Place all the access tokens in the `.env` file and upload it to the working directory (if you are running this notebook locally, you can change the path to fit your working directory). Please use the following format:


```
HF_AUTH= "Hugging Face Authentication Key"
PINECONE_API_KEY="Pincone API Key"
PINECONE_ENVIRONMENT="Pinecone Environment"
```

Run the cell below to load the access tokens into the environment variables.

In [1]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(raise_error_if_not_found=True))



## Subtask 1.1: Data Preparation



We need a collection of documents to perform our retrieval on. To make it closer to your final project, you will be downloading and using a subset of the LangChain documentation. We get some of the `.html` files located on the site. The code below will download all HTML files from the links on the webpage into a `docs` directory. `-l1` limits the download to only the first level of depth.


 The docs are going to be used as input text for answering questions that a normal language model might not be aware of (LangChain docs is not necessarily part of its training data of Llama2). We can use LangChain itself to process these docs. Use the [ReadTheDocsLoader](https://python.langchain.com/docs/integrations/document_loaders/readthedocs_documentation) to load the docs from the `docs` folder.

 At the time of creating this notebook, there  `423` documents were downloaded. However, since the documentation is being updated regularly this number might be different for you.

Let's take a look at one of the documents. You see that LangChain has created a `Document` object. Look at the example below and fill in the cells to print out the text content and URL of the page (the URL of the page should starts with `https://`).

As you can imagine the documents can be long and if multiple of them are required as context to answer questions, we need to take the document lengths into account.
This is due to the fact that language models do not have unlimited context span. In our case, we plan to use Llama2 for this project, where the maximum token limit is 4096. This limit is not only the input but also takes the generated output into account, moreover, you need to leave room for the query and instructions as well. Therefore, it is important to chunk the longer documents into smaller-sized fragments.

Based on your use case and how many contexts you plan to feed into the model the length of these fragments will differ.
In this case, we choose to assign 2000 tokens to context and choose to generate the answer from 5 context fragments, which leaves us with 400 tokens per context fragment as the maximum chunk size.

To count the number of tokens in a chunk, we need to load the correct tokenizer for Llama2. Fill the code cell below to load the correct tokenizer and use it to complete the function that counts the number of tokens per given chunk.

**Hint:** you need to use your Hugging Face authentication token to load the tokenizer.

In [2]:
#If you get an error here during the first import from the `transformers` package, restart the kernel and try again.
#### your code ####
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", token=os.getenv("HF_AUTH"))
#### your code ####

c:\Users\domin\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Count the number of tokens for all documents and use it to compute minimum, maximum, and average token count statistics across all documents. Depending on how the documentation is updated by the time you run the cell below the numbers might slightly differ.

Now we will use LangChain's built-in chunking functionality to split the text into smaller chunks. LangChain offers a variety of text splitters that you can check out [here](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.text_splitter).
Use the general-purpose splitter that splits text by recursively looking at characters. Use this class to split the text into 400 token-sized chunks, where the length of each chunk is computed based on the `token_len` function. The length is not the only criterion for splitting, if any of these separators `'\n\n', '\n', ' ', ''` is encountered, we will have a new chunk.
Since splitting only based on maximum length might result in incoherent chunks for every consecutive chunk, let the chunk overlap by 50 tokens. This way,  we preserve some of the previous context while chunking.

The next step is to apply the splitting function to all the documents in our corpus and to save our chunks in a logical way. We also want to assign a unique ID to each chunk so we know which part of the documentation they come from. In the end, the corpus should be transformed into a list of dictionaries of the following format:


```
[
    {
        "id": "glossary-0",
        "text": "first chunk of the document glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    },
    {
        "id": "glossary-1",
        "text": "second chunk of glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    }
    ...
]
```

Construct the IDs by taking the name of the page before the suffix `.html` and appending a chronological number indicating which chunk it is.


For the next steps, we require a `DataFrame`.

#### ${\color{red}{Comments\ 1.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.2: Document Embedding Pipeline


In this task, we initialize the embedding pipeline to transform the chunks into vector embeddings using Hugging Face and LangChain. These embeddings are used for similarity search between the query and the chunks to retrieve the most relevant chunks.
  We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding, which is a rather small model that you can easily run on Colab. Initialize the model using `HuggingFaceEmbeddings` to use Hugging Face via Langchain. The encoding batch size should be 32, and make sure that the model is placed on the correct device, otherwise, this can take a long time.

In [3]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
from tqdm import tqdm

Embed the example documents using the model you created and check the output.
The output should be a list of lists, containing the embeddings.

Now we use the embedding pipeline created above to store the embeddings in a Pinecone vector index. First, lets setup the Pinecone environment, collect your API key and environment name from the environment variables, and initiate Pinecone with them.

Initialize the index `rag-assignment` inside Pinecone. Use the cosine similarity as similarity metric. Keep in mind that if you run this multiple times on a free tier, where only one index is allowed, you need to remove the index created to make room for a new one (Pinecone index gets archived automatically after 14 days of inactivity).

Lets take a look at the index you created. As of now the index should be empty but have the correct embedding dimension.

Process the dataset in batches of `32` and push the vectors to the Pinecone index. Your index should include the IDs and embeddings for each chunk. As metadata, pass the original text as `text` and the URL as `source` (no need to add the `https`). We use this metadata later to retrieve the original text.

Now if we look at the index statistics we should have vectors of dimension `384`.

#### ${\color{red}{Comments\ 1.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.3: Text Generation Pipeline


So far we have our index ready and a way to find the most similar chunks to our query. Now, we need a way to generate the answer from the retrieved chunks. For this purpose, we use the `text-generation` pipeline from Hugging Face (refer to the Hugging Face [tutorial](https://moodle.uni-heidelberg.de/pluginfile.php/1286642/mod_resource/content/1/HuggingFace.ipynb)) and load it into LangChain using a wrapper.

In [4]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'TheBloke/Llama-2-7B-GGUF' # 'meta-llama/Llama-2-13b-chat-hf'

Quantization techniques reduce memory and computational costs by representing weights and activations with lower-precision data types like 8-bit integers (int8). This enables loading larger models you normally wouldn’t be able to fit into memory, and thus speeds up inference.
To make the process of model quantization more accessible, Hugging Face has seamlessly integrated with the [Bitsandbytes](https://huggingface.co/docs/accelerate/usage_guides/quantization) library.

Define a config from `Bitsandbytes` that enables 4-bit quantization and set the nested quantization to `true`. This changes the datatype from float 32 (default) to normalized float 4 datatype to contain 4 bits of information.
Additionally, add a compute type to store weights in 4-bits, but the computation to happen in 16-bit (bfloat16).
Moreover, set the `bnb_4bit_use_double_quant` to true, which uses a second quantization after the first one to save an additional 0.4 bits per parameter.
Refer to [here](https://huggingface.co/docs/transformers/main_classes/quantization) for more information.

In [5]:
### your code ###
bitsAndBytes_config = transformers.BitsAndBytesConfig(
    #bnb_4bit_quant_type='nf4',
    load_in_8bit=True,
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=bfloat16
)
### your code ###

Use your Hugging Face token to load the correct model configuration using the `transformers` library.

In [6]:
### your code ###
model_config = transformers.AutoConfig.from_pretrained(model_id, use_auth_token=os.getenv("HF_AUTH"))
### your code ###


c:\Users\domin\miniconda3\lib\site-packages\transformers\models\auto\configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Load the model for text generation (pay attention to the model type) using the configuration file you have defined, with the specified quantization, and set the `trust_remote_code` flag to `true`. Another flag that is useful for large mode is  `device_map="auto"`. By setting this flag, Accelerate will determine where to put each layer to maximize the use of GPUs and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM).

It will take a while for the model to download.

In [10]:
#Loading the model will take some time, (roughly 5 min)
from ctransformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-GGUF", model_file="llama-2-7b.Q4_K_M.gguf", model_type="llama", gpu_layers=1)
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     config=model_config,
#     trust_remote_code=True,
#     device_map="auto",
#     #quantization_config=bitsAndBytes_config,
#     token=os.getenv("HF_AUTH")
# )
### your code ###
#model.eval()# we only use the model for inference
print(f"Model loaded ")

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


FileNotFoundError: Could not find module 'C:\Users\domin\miniconda3\Lib\site-packages\ctransformers\lib\cuda\ctransformers.dll' (or one of its dependencies). Try using the full path with constructor syntax.

You can even check the memory footprint of your model using the `get_memory_footprint` method.


In [18]:
model.get_memory_footprint()

AttributeError: 'LLM' object has no attribute 'get_memory_footprint'

The next thing we need to do is initialize a `text-generation` pipeline with Hugging Face that uses the Llama2 model to generate some text, given some input. We will then use this pipeline inside LangChain to build our question-answering system.
`text-generation` pipeline generates text from a language model conditioned on a given input. The pipeline is similar to other Hugging Face pipelines and requires two things that we must initialize:

1.   A language model, in this case, it will be `meta-llama/Llama-2-13b-chat-hf`.
2.   A tokenizer for the language model.

LangChain expects the full-text outputs, therefore set the `return_full_text` to true. You can also pass additional generation parameters to the model.
Since we want the questions to be answered mainly based on the retrieved chunks, let's set the model temperature to a low value of 0.01 to reduce randomness. Additionally, add a repetition penalty of 1.1 to stop the model from repeating itself and the maximum number of generation tokens to 512.

In [9]:
### your code ###
print(model("pimmel"))
# Docu: https://huggingface.co/docs/transformers/v4.37.1/main_classes/text_generation
# generate_text = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device_map="auto",
#     return_full_text=True,
#     temperature = 0.01,
#     repetition_penalty = 1.1,
#     max_new_tokens = 512
# )
### your code ###

KeyboardInterrupt: 

We provide the language model a general question to make sure our pipeline is working correctly.

In [11]:
sample_input="Explain to me the difference between alligator and crocodile."
### your code ###

generated_text=generate_text(sample_input)
### your code ###
print(generated_text)

KeyboardInterrupt: 

Use the LangChain Hugging Face wrapper, as subset of [LLM chain](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) to create an interface for the text generation pipeline.

In [ ]:
### your code ###
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)
### your code ###

To confirm that it works the same way, use the sample input to generate text using the llm chain. The input should be passed as the `prompt` to the language model.

In [ ]:
### your code ###

# Alternative version on how to prompt the model just in case:
# (we didn't know what was expected in the assignment..)
# from langchain.prompts import PromptTemplate
# template = """Question: {question}

# Answer: Let's think about this."""
# prompt = PromptTemplate.from_template(template)
# chain = prompt | llm
# chain.invoke({"question": sample_input})


llm(sample_input)
### your code ###

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nAlligators and crocodiles are both large, carnivorous reptiles that live in wetlands and rivers, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have more prominent bumps on their skin, especially on their backs.\n2. Habitat: Alligators prefer freshwater habitats such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Behavior: Alligators are generally less aggressive than crocodiles and tend to avoid confrontations with humans. Crocodiles, on the other hand, are known for their aggressive behavior and have been responsible for many human attacks and f

#### ${\color{red}{Comments\ 1.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.4: Question Answering Chain


For Retrieval Augmented Generation (RAG) in LangChain, we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object.

`RetrievalQA` is a method for question-answering tasks, utilizing an index to retrieve relevant documents or text chunks, it is suitable for straightforward Q&A applications.

`RetrievalQAWithSourcesChain` is an extension of RetrievalQA that chains together multiple sources of information, providing context and the source for answers.

 For both of these, we need an LLM and a Pinecone index. For LangChain to be able to use the Pinecone index, we need to initialize it through the LangChain vector store.

 **Hint**: You need to explicitly tell the vector storage where to find the original text.

In [ ]:
from langchain.vectorstores import Pinecone
### your code ###
vectorstore = Pinecone.from_existing_index(
    "rag-assignment",
    embed_model,
    'text'
)
### your code ###

Let's try a query that is specific to the LangChain documentation and see which chunks are relevant. Use the vector storage defined above to find the top-3 chunks related to the given query.

In [ ]:
query = 'what is a LangChain Agent?'
### your code ###
top_chunks = vectorstore.similarity_search(query, k=3)
top_chunks
### your code ###

[Document(page_content='langchain 0.1.3¶\nlangchain.agents¶\nAgent is a class that uses an LLM to choose a sequence of actions to take.\nIn Chains, a sequence of actions is hardcoded. In Agents,\na language model is used as a reasoning engine to determine which actions\nto take and in which order.\nAgents select and use Tools and Toolkits for actions.\nClass hierarchy:\nBaseSingleActionAgent --> LLMSingleActionAgent\n                          OpenAIFunctionsAgent\n                          XMLAgent\n                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent\nBaseMultiActionAgent  --> OpenAIMultiFunctionsAgent\nMain helpers:\nAgentType, AgentExecutor, AgentOutputParser, AgentExecutorIterator,\nAgentAction, AgentFinish\nClasses¶\nagents.agent.Agent\n[Deprecated]  Agent that calls the language model and deciding the action.\nagents.agent.AgentExecutor\nAgent that is using tools.\nagents.agent.AgentOutputParser\nBase class for parsing agent output into agent acti

Now use the `vectorstore` and `llm` to initialize the `RetrievalQA` object, which showcases question answering over an index. `RetrievalQA` is a document chain, these are useful for summarizing documents, answering questions about documents, extracting information from documents, and more. All such chains operate with 4 different chain types:


1.   `stuff`: it takes a list of documents, inserts them all into a prompt, and passes that prompt to an LLM.
2.   `refine`: it constructs a response by looping over the input documents and iteratively updating its answer. It is well-suited for tasks that require analyzing more documents than can fit in the model’s context.
3. `map_reduce`:  it first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combined documents chain to get a single output (the Reduce step).
4. `map_re_rank`: it runs an initial prompt on each document that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest-scoring response is returned.

For this assignment, we focus only on the first type. Make sure to set the `verbose` to `true`, so we can see the different stages of processing that happens while answering a question (you might need to set this parameter more than once). As mentioned before, we want our retrieve to input top-5 most similiar chunks to the query to generate an answer.

In [ ]:
from langchain.chains import RetrievalQA
### your code ###
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

### your code ###
query='what is a LangChain Agent?'

First, we try to answer the question only using Llama2. As you see the answer is not convincing as it does not have access to the LangChain documentation.

In [ ]:
llm(query)

'\n\nA LangChain Agent is an intelligent agent that uses natural language processing (NLP) and machine learning (ML) techniques to assist users in finding relevant information on the web. It is designed to help users navigate the vast amount of information available online by providing personalized recommendations and answers to their questions.\n\nThe name "LangChain" refers to the idea of linking together different languages and knowledge sources to create a comprehensive and coherent view of the world. The agent is able to understand and respond to user queries in multiple languages, and it can draw upon a wide range of sources, including text, images, videos, and other forms of media, to provide accurate and relevant results.\n\nSome of the key features of a LangChain Agent include:\n\n1. Natural Language Processing (NLP): The agent is able to understand and interpret natural language queries, allowing users to ask questions in everyday language.\n2. Machine Learning (ML): The agen

Now use the Pipeline from above and see how the answer changes.

In [ ]:
### your code ###
rag_pipeline(query)
### your code ###


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is a LangChain Agent?',
 'result': ' A LangChain Agent is a piece of software that uses a Language Model (LLM) to decide what actions to take. It is part of the LangChain project, which aims to provide a platform for building and deploying conversational AI models. The agent is driven by an LLM chain, which includes a series of language models that are trained on different tasks. The agent can use tools and toolkits to perform actions, and it can also parse agent output to determine which actions to take next.'}

#### ${\color{red}{Comments\ 1.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.5: Conversational Retrieval Chain




We can also extend our retrieval chain to be able to remember the previous questions and answer the current question by looking at the previous context.
The important part of a conversational model is conversation memory, which transforms the stateless language model to be able to remember previous interactions, e.g., similiar to ChatGPT. In this subtask, we will use LangChain to create a conversational memory.


To implement the memory we use `ConversationalRetrievalChain`.
This chain takes in chat history (a list of messages) and new questions and then returns an answer to that question. The algorithm for this chain consists of three parts:

1. Use the chat history and the new question to create a new question that contains the information from the previous context.

2. This new question is passed to the retriever and relevant documents are returned.

3. The retrieved documents are passed to an LLM to generate a final response.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
chat_history = []

### your code ###
qa_conversation = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever)

result = qa_conversation({"question": query, "chat_history": chat_history})
### your code ###


In [ ]:
result

{'question': 'what is a LangChain Agent?',
 'chat_history': [],
 'answer': ' A LangChain Agent is a piece of software that uses a Language Model (LLM) to decide what actions to take. It is part of the LangChain project, which aims to provide a platform for building and deploying conversational AI models. The agent is driven by an LLM chain, which includes a series of language models that are trained on different tasks. The agent can use tools and toolkits to perform actions, and it can also parse agent output to determine which actions to take next.'}

In [ ]:
result["answer"]

' A LangChain Agent is a piece of software that uses a Language Model (LLM) to decide what actions to take. It is part of the LangChain project, which aims to provide a platform for building and deploying conversational AI models. The agent is driven by an LLM chain, which includes a series of language models that are trained on different tasks. The agent can use tools and toolkits to perform actions, and it can also parse agent output to determine which actions to take next.'

Change the chat history to contain the previous question and answer pair and ask a follow-up question.  

In [ ]:
follow_up="What are tools and toolkits?"

### your code ###
chat_history = [(query, result["answer"])]
result = qa_conversation({"question": follow_up, "chat_history": chat_history})
### your code ###

This is the previous context that was fed in alongside the new question.

In [ ]:
chat_history

[('what is a LangChain Agent?',
  ' A LangChain Agent is a piece of software that uses a Language Model (LLM) to decide what actions to take. It is part of the LangChain project, which aims to provide a platform for building and deploying conversational AI models. The agent is driven by an LLM chain, which includes a series of language models that are trained on different tasks. The agent can use tools and toolkits to perform actions, and it can also parse agent output to determine which actions to take next.')]

The current question is answered by knowing that the tools and toolkits are referring to a LangChain Agent, which was part of the previous question.

In [ ]:
result['answer']

'  Tools and toolkits are used to perform specific actions or tasks within a LangChain Agent. They can be thought of as "skills" or "abilities" that the agent possesses, allowing it to interact with its environment in various ways. For example, an agent might have a "document-reading" tool that allows it to analyze and understand text documents, or a "question-answering" tool that enables it to respond to user queries. Toolkits are collections of related tools that can be used together to accomplish more complex tasks.'

#### ${\color{red}{Comments\ 1.5}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 2: Advanced RAG Techniques and Evaluation (4 + 5 = 9 points)**

Now that you have successfully implemented your first RAG system, we dive into more advanced techniques and learn how to evaluate your methods using metrics you learned during the lecture. We focus on evaluation with an already annotated dataset. To this end, we load a small subset of [NarrativeQA](https://huggingface.co/datasets/narrativeqa), which is an English-language dataset of stories and corresponding questions designed to test reading comprehension, especially on long documents. We only load 30 samples from the data, as you will see in the upcoming sections, answer generation takes quite some time. In actual setting, it is advised to use a much larger set to obtain statistically significant results.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("satyaalmasian/narrativeqa_subset", split="train[:30]")
len(dataset)

30

Since we already used our free index in Pinecone for the previous task, we use Chroma, an open-source vector database, instead. As opposed to Pinecone, Chroma creates a collection on your machine.

In [ ]:
from langchain.docstore.document import Document
documents=[doc["text"] for doc in dataset["document"]]
questions=[quest for quest in dataset["question"]]
answers=[ans for ans in dataset["answers"]]
documents=list(set(documents))

In [ ]:
docs = [Document(page_content=doc, metadata={"source": "local"}) for doc in documents]
for doc in docs:
  print(doc)

page_content='ï»¿The Project Gutenberg EBook of A Voyage to Arcturus, by David Lindsay\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: A Voyage to Arcturus\n\nAuthor: David Lindsay\n\nPosting Date: September 17, 2008 [EBook #1329]\nRelease Date: May, 1998\n[Last updated: June 28, 2012]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK A VOYAGE TO ARCTURUS ***\n\n\n\n\nProduced by An Anonymous Volunteer\n\n\n\n\n\nA VOYAGE TO ARCTURUS.\n\nBy David Lindsay\n\n\nContents:\n\n     1   The Seance\n     2   In the Street\n     3   Starkness\n     4   The Voice\n     5   The Night of Departure\n     6   Joiwind\n     7   Panawe\n     8   The Lusion Plain\n     9   Oceaxe\n     10  Tydomin\n     11  On Disscourn\n     12  Spadevil\n     13  The Wombflash Forest

The number of documents is smaller  than the number of questions and answers and each document is used as a reference for multiple questions:

In [ ]:
print(len(docs))
print(len(questions))

2
30


##Subtask 2.1: Build Contextual Compression in LangChain

Let's split our documents using the TextSplitter from Task 1 and embed them inside the Chroma database with the embedding model of the previous task.

In [ ]:
### your code ###
chunks = text_splitter.split_documents(docs)
print(len(chunks))
for chunk in chunks:
  print(chunk)
### your code ###

534
page_content='ï»¿The Project Gutenberg EBook of A Voyage to Arcturus, by David Lindsay\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: A Voyage to Arcturus\n\nAuthor: David Lindsay\n\nPosting Date: September 17, 2008 [EBook #1329]\nRelease Date: May, 1998\n[Last updated: June 28, 2012]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK A VOYAGE TO ARCTURUS ***\n\n\n\n\nProduced by An Anonymous Volunteer\n\n\n\n\n\nA VOYAGE TO ARCTURUS.\n\nBy David Lindsay\n\n\nContents:' metadata={'source': 'local'}
page_content="Produced by An Anonymous Volunteer\n\n\n\n\n\nA VOYAGE TO ARCTURUS.\n\nBy David Lindsay\n\n\nContents:\n\n     1   The Seance\n     2   In the Street\n     3   Starkness\n     4   The Voice\n     5   The Night of Departure\n     6   Joiwind\

In [ ]:
from langchain.vectorstores import Chroma
### your code ###
chromadb = Chroma.from_documents(chunks, embed_model)
chroma_retriever = chromadb.as_retriever(search_kwargs={"k": 3})
### your code ###

In [ ]:
first_question = questions[2]['text']
print(f"First question in the set: {first_question}")
r_docs = chroma_retriever.get_relevant_documents(first_question)
for r_doc in r_docs:
  print(r_doc, end="\n")

First question in the set: Why do more students tune into Mark's show?
page_content="PTA. Parent #4 - I work with teenage gangs in the city I say we go after this guy.\n\n<Paige walks in>\n\nPaige - My name is Paige Woodward and I have something to say to you people. People \nare saying that Harry is introducing bad things and encouraging bad things. But it seems \nto me that these things were already here. My god why don't you people listen? He's \ntrying to tell you something is wrong with this school. Half the people that are here are on \na probation of some kind. We are all really scared to be who we really are. I am not \nperfect. I've just been going through the motions of being perfect, and inside I'm \nscreaming.\n\nCreswood - Paige, you were a model student.\n\n<Paige walks out were the press await>\n\nReporter #2 - Do you know who he is? Are you prepared to do anything he says?\n\nPaige - <Shouting into the camera> Can you hear me? Don't listen to them, don't listen to \nany

First, make a simple RAG pipeline that works on top of the Chroma retriever. This retriever should be similar to the previous task. However, since we want to use it for a large number of questions, remove the `verbose` parameters.

In [ ]:
from langchain.chains import RetrievalQA
### your code ###
rag_simple = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_retriever
)
### your code ###

We look at an example question and compare the answer by RAG to the gold answer from the dataset. Note that the answers can contain multiple lines.

In [ ]:
question = questions[2]['text']
rag_simple(question) #ignore the warning

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'query': "Why do more students tune into Mark's show?",
 'result': ' Because they want to hear something real and authentic, rather than the superficial perfection that is expected of them at school.'}

In [ ]:
answers[2]

[{'text': 'Mark talks about what goes on at school and in the community.',
  'tokens': ['Mark',
   'talks',
   'about',
   'what',
   'goes',
   'on',
   'at',
   'school',
   'and',
   'in',
   'the',
   'community',
   '.']},
 {'text': 'Because he has a thing to say about what is happening at his school and the community.',
  'tokens': ['Because',
   'he',
   'has',
   'a',
   'thing',
   'to',
   'say',
   'about',
   'what',
   'is',
   'happening',
   'at',
   'his',
   'school',
   'and',
   'the',
   'community',
   '.']}]

Apply the `rag_simple` pipeline to all the question in your corpus and accumulate the answers. **It should take around 10 minutes on a T4 GPU on Colab**.

In [ ]:
simple_answers=[]
### your code ###
for question in tqdm(questions):
  question = question["text"]
  answer = rag_simple(question)["result"]
  simple_answers.append(answer)
  print(f"Question: {question} --> Answer: {answer}")

print(len(simple_answers))
print(simple_answers)
### your code ###

  3%|▎         | 1/30 [00:17<08:37, 17.84s/it]

Question: Who is Mark Hunter? --> Answer:  Mark Hunter is Happy Harry Hardon.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  7%|▋         | 2/30 [00:38<09:11, 19.70s/it]

Question: Where does this radio station take place? --> Answer:  Based on the dialogue provided, it appears that the radio station takes place in a dystopian or post-apocalyptic society, as the characters mention "tracktones" and refer to the "rankness" in the air. However, without more context, it is difficult to determine the exact location of the radio station.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 10%|█         | 3/30 [01:11<11:33, 25.70s/it]

...

Question: Who is chasing Mark and Nora in the jeep? --> Answer:  The F.C.C. (Federal Communications Commission) vans are chasing Mark and Nora in the jeep.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 97%|█████████▋| 29/30 [10:17<00:22, 22.39s/it]

Question: What are the students doing when Mark and Nora drive up? --> Answer:  Based on the dialogue provided, the students are not doing anything specific when Mark and Nora drive up. The conversation between Mark and Nora focuses on their own thoughts and feelings, rather than any actions they are taking. Therefore, there is no clear answer to this question.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 30/30 [10:38<00:00, 21.28s/it]

Question: Who does Maskull accept an invitation from? --> Answer:  Based on the text, Maskull accepts an invitation from Sullenbode.
30
[' Mark Hunter is Happy Harry Hardon.', ' Based on the dialogue provided, it appears that the radio station takes place in a dystopian or post-apocalyptic society, as the characters mention "tracktones" and refer to the "rankness" in the air. However, without more context, it is difficult to determine the exact location of the radio station.', " Based on the dialogue provided, it appears that more students tune into Mark's show because they are drawn to his rebellious attitude and desire to break free from the constraints of their perfect image. They may also be attracted to the idea of staying up late and listening to music that is not typically allowed in their strict environment. Additionally, Mark's show may offer a sense of community or belonging for students who feel isolated or disconnected from their peers.", ' Happy Harry Hardon.', ' Based on 

Libraries such as LangChain and [Llamaindex](https://www.llamaindex.ai/) provide a variety of retrieval strategies for building a RAG system. In this subtask, you will use one of these variations called **contextual compression**. This method aims to extract only the relevant information from documents, reducing the need for expensive language model calls and improving response quality. Contextual compression consists of two parts:


1.  **Base retriever:** retrieves the initial set of documents based on the query. This is similar to the retriever from the previous task.
2.   **Document compressor:** processes these documents to extract the relevant content. We use `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter
from langchain.llms import OpenAI

### your code ###
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=chroma_retriever
)
### your code ###

Let's take a look at an example of compression retriever works.

In [ ]:
print("First question in the set:",questions[2]['text'])
normal_docs = chroma_retriever.get_relevant_documents(questions[2]['text'])
for normal_doc in normal_docs:
  print(normal_doc)

compressed_docs = compression_retriever.get_relevant_documents(questions[2]['text'])
for compressed_doc in compressed_docs:
  print(compressed_doc)

First question in the set: Why do more students tune into Mark's show?
page_content="PTA. Parent #4 - I work with teenage gangs in the city I say we go after this guy.\n\n<Paige walks in>\n\nPaige - My name is Paige Woodward and I have something to say to you people. People \nare saying that Harry is introducing bad things and encouraging bad things. But it seems \nto me that these things were already here. My god why don't you people listen? He's \ntrying to tell you something is wrong with this school. Half the people that are here are on \na probation of some kind. We are all really scared to be who we really are. I am not \nperfect. I've just been going through the motions of being perfect, and inside I'm \nscreaming.\n\nCreswood - Paige, you were a model student.\n\n<Paige walks out were the press await>\n\nReporter #2 - Do you know who he is? Are you prepared to do anything he says?\n\nPaige - <Shouting into the camera> Can you hear me? Don't listen to them, don't listen to \nany

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


page_content='* "stay on, stay hard"\n* "Talk Hard"' metadata={'source': 'local'}
page_content='* "stay on, stay hard"\n* "Talk Hard"' metadata={'source': 'local'}
page_content='* "stay on, stay hard"\n* "Talk Hard"' metadata={'source': 'local'}
page_content='* "stay on, stay hard"\n* "Talk Hard"' metadata={'source': 'local'}


Look at the output and try out several different questions by yourself. Does the compressed output make sense?

Compare this to the previous **simple** approach. Which one, in your opinion, is better?

**Answer:**

The compressed output somewhat makes sense but is very short and not elaborate. Yet, you have to keep in mind that this is only an intermediary step because it is consumed by the LLM to produce more verbose text again. Still, we prefer the uncompressed variant.

 (Note: It returns the same multiple times because the document retriever collects the same documents multiple times due to unknown reasons. We verified the implementation and it is correct, yet the retrieval result seems off.)

Finally, we use the new retriever with the Llama2 model from the previous task to create the context compressor RAG pipeline. The code below should be similiar to what you did in the previous task. Once again, make sure to turn off the `verbose` argument.

In [ ]:
### your code ###
from langchain.chains import RetrievalQA

rag_compressor = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever
)
### your code ###


In [ ]:
question = questions[2]['text']
rag_compressor(question)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


{'query': "Why do more students tune into Mark's show?",
 'result': ' Because he tells them to "stay on, stay hard" and they think it means something cool or important.'}

Now we can use the pipeline to generate answers for all the questions in our dataset. **It should take around 20 minutes on a T4 GPU on Colab.**

In [ ]:
compressor_answers=[]
### your code ###
for question in tqdm(questions):
  question = question["text"]
  answer = rag_compressor(question)["result"]
  compressor_answers.append(answer)
  print(f"Question: {question} --> Answer: {answer}")

print(len(compressor_answers))
print(compressor_answers)
### your code ###

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass a

Question: Who is Mark Hunter? --> Answer:  Mark Hunter is a character in the novel "Happy Harry Hardon" by John Updike.


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


Question: Where does this radio station take place? --> Answer:  This radio station takes place right around here, everywhere I look it seems like the coolest place to be!


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


...

Question: Who is chasing Mark and Nora in the jeep? --> Answer:  The F.C.C. (Federal Communications Commission) is chasing Mark and Nora in the jeep.


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


Question: What are the students doing when Mark and Nora drive up? --> Answer:  Based on the context provided, it appears that the students are participating in a school project where they are identifying plants and animals in their neighborhoods. The phrase "first stage personal identification" suggests that the students are learning about the different species in their area and how to identify them. The statement "I have neighbors, stop!" may indicate that the students are working in groups or teams to collect data and take measurements. Finally, the phrase "Yes I can" suggests that the students are confident in their ability to complete the project successfully.


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.


Question: Who does Maskull accept an invitation from? --> Answer:  Based on the given text, Maskull accepts an invitation from Haunte.
30
[' Mark Hunter is a character in the novel "Happy Harry Hardon" by John Updike.', ' This radio station takes place right around here, everywhere I look it seems like the coolest place to be!', ' Because he tells them to "stay on, stay hard" and they think it means something cool or important.', ' No one deserves to die by suicide. Suicide is a permanent solution to temporary problems. It is not a sign of weakness, but rather a sign that someone is struggling and needs help. If you or someone you know is struggling with thoughts of suicide, it is important to seek professional help as soon as possible. There are many resources available to support those in crisis, including the National Suicide Prevention Lifeline (1-800-273-TALK) and online resources such as the American Foundation for Suicide Prevention and the National Alliance on Mental Illness.',

#### ${\color{red}{Comments\ 2.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

##Subtask 2.2. Evaluate

Since we have access to ground truth answers, we can use various evaluation metrics from the literature. In this task, we explore three metrics:


1.   **BLEU:** BLEU score stands for Bilingual Evaluation Understudy and is a precision-based metric developed
for evaluating machine translation. BLEU scores a candidate by computing the
number of n-grams in the candidate that also appear
in a reference. The n can vary, in this task we compute for n=4.
2.   **ROUGE:** ROUGE score stands for Recall-Oriented Understudy for Gisting Evaluation and is an F-measure metric designed for
evaluating translation and summarization. There are a number of variants of ROUGE.
3. **BERTScore:** BERTScore first obtains BERT representation of each word in the candidate and reference by feeding the candidate
and reference through a BERT model separately.
An alignment is then computed between candidate
and reference words by computing pairwise cosine
similarity. This alignment is then aggregated in to
precision and recall scores before being aggregated
into a (modified) F1 score that is weighted using
inverse-document-frequency values.

Luckily, Hugging Face has an implementation for all these metrics. Use the `evaluate` library to load the metrics.

Use the loaded metrics to compare the RAG pipelines from the previous subtask.

In [ ]:
import evaluate
### your code ###
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
### your code ###

As seen in the previous subtask, the answers can contain multiple lines. To be able to compare the output of our systems to the gold answers, merge the multiple answers into a single string.

In [ ]:
answers_merged = []
### your code ###
for answer in answers:
  merged_answer = ""
  for partial_answer in answer:
    merged_answer += f" {partial_answer['text']}"
  answers_merged.append(merged_answer)

### your code ###
print(answers_merged)
print(len(answers_merged))

[' He is a high school student in Phoenix. A loner and outsider student with a radio station.', " It takes place in Mark's parents basement.  Phoenix, Arizona", ' Mark talks about what goes on at school and in the community. Because he has a thing to say about what is happening at his school and the community.', ' Malcolm. Malcolm.', ' She jams her medals and accolades.  Her award medals', " He dismantles it and attaches it to his mother's jeep. Dismantle it.", ' He tells them to make their own future. That they should make their own future because the world belongs to them.', ' Mark and Nora. Mark and Nora.', ' It causes trouble.  It causes much trouble in the community.', " Parent's Basement At the basement of his home", ' His Radio station  His unauthorized radio station.', ' Everybody Knows "Everybody Know\'s"', ' Fellow Student a fellow student', " Malcom' s suicide To confront him after Malcolm commits suicide.  ", ' Do something about their problems. To do something about their 

Compute the BLUE score for the simple RAG and compressor RAG.

In [ ]:
### your code ###
bleu_simple = bleu.compute(predictions=simple_answers, references=answers_merged)
bleu_compressor = bleu.compute(predictions=compressor_answers, references=answers_merged)
### your code ###
print("Simple system:")
print(bleu_simple)
print("Compressor:")
print(bleu_compressor)

Simple system:
{'bleu': 0.0, 'precisions': [0.09442548350398179, 0.004711425206124852, 0.001221001221001221, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.9695945945945947, 'translation_length': 879, 'reference_length': 296}
Compressor:
{'bleu': 0.0, 'precisions': [0.0812720848056537, 0.006105006105006105, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.8682432432432434, 'translation_length': 849, 'reference_length': 296}


What does the elements below in the output of the BLEU impelementation in Hugging Face mean? (do not copy and paste the documentation but write the implications behind each element!).


1.   **precisions:** The precisions attribute in the BLEU output represents the precision scores for different n-gram sizes (so in this case unigrams/1-gram, bigrams/2-gram, trigrams/3-gram, 4-gram as n=4). Each precision value reflects how well the machine-generated translation aligns with the reference translations at different n-gram levels. We can observe that the precision score drops to zero for larger n, implying that those n-grams cannot be found in the text. That is somewhat expected, because we are not extracting the text from the retrieved documents, but rather generate new content using the Llama model.
2.   **brevity_penalty:** The brevity_penalty attribute accounts for the length of the machine-generated answer compared to the average length of the reference answer. It penalizes overly short answers that might otherwise receive higher BLEU scores. Since the attribute is set to 1.0 in both cases, it indicates that the answers are not too short.
3.   **translation_length:** The translation_length attribute describes the total length of the machine-generated answers.
4.   **reference_length:** The reference_length attribute describes the total length of the reference answers.
5.   **length_ratio:** The length_ratio attribute describes the ratio between translation_length and reference_length.
We can see that in both cases, the translation_length is way larger than the reference_length. Hence, we can attest that the model generates long answers instead of short and consise ones. In fact, the answers for the compressed RAG are slighly shorter, but not significantly than for the simple RAG. That could indicate that compressing does not impede answering ability too much and at least leads to similar-lengthed answers - yet, nothing is said about the quality just by that.



**Answer:**


1.   **precisions:** precision of n-grams, which is calculated as the number of n-grams that appear in both the machine-generated translation and the reference translations divided by the total number of n-grams in the machine-generated translation.
2.   **brevity_penalty:** is a penalty term that adjusts the score for translations that are shorter than the reference translations. It is calculated as min(1, (reference_length / translation_length)). It essentially penalizes generated translations that are too short compared to the closest reference length with an exponential decay.
3.   **translation_length:**   is the total number of words in the machine-generated translation.
4.   **reference_length:**  is the total number of words in the reference translations.
5. **length_ratio:** ratio of the 3 and 4.

In [ ]:
### your code ###
rouge_simple = rouge.compute(predictions=simple_answers, references=answers_merged)
rouge_compressor = rouge.compute(predictions=compressor_answers, references=answers_merged)
### your code ###
print("Simple system:")
print(rouge_simple)
print("Compressor:")
print(rouge_compressor)

Simple system:
{'rouge1': 0.11642546287651759, 'rouge2': 0.006127637935418256, 'rougeL': 0.10110030756743321, 'rougeLsum': 0.10233186928011667}
Compressor:
{'rouge1': 0.12291840635579375, 'rouge2': 0.008548318433375906, 'rougeL': 0.10835590214537304, 'rougeLsum': 0.1080779807552118}


**What is the difference in variants of ROUGE (ROUGE-N, ROUGE-L, ROUGE-SUM)?**

ROUGE in its core is a set of metrics used for evaluating the quality of automatic summaries. It measures the overlap between automatically generated summaries and reference summaries. There are several variants of ROUGE, each focusing on different aspects of summarization:

**(1) ROUGE-N**: This variant evaluates the overlap of n-grams between the generated summary and the reference summary using an F1-score (and hence precission and recall) calculated from the overlap. In this case unigrams/1-grams (rouge1) and bigrams/2-grams (rouge2) have been inspected. ROUGE-N is sensitive to word matching and hence is used to evaluate content overlap and grammatical correctness. Since it is word-sensitive, the rouge2 score is lower than the rouge1 score. That is because it is more likely to find matches of only one word than matches of two consecutive words as no extraction is performed but rather reprasing of the retrieved documents.

**(2) ROUGE-L**: This variant determines the longest common subsequence (LCS) between the generated summary and the reference summary and calculates an F1-score (and hence precision and recall) from that. ROUGE-L is designed to be more robust to reordering of words and hence can be used to evaluate semantic similarity and content coverage.

**(3) ROUGE-S**: This variant determins the overlap of skip-grams between the generated summary and the reference summary and calculates an F1-score (and hence precission and recall) from that. Since skip grams are allowed to have a word gap and leave out words in between, it is useful for evaluating textual coherence and semantic similarity.

Conclusively, all of the ROUGE variants focus on different aspects of text summarization. ROUGE-N focussed on raw word sequence matching, ROUGE-L focusses on content coverage by searching for the longest textual "common denominator" and ROUGE-S focusses on textual coherence by looking at connected word pairs that may have interleaved words.

**Answer:**

ROUGE measures the similarity between the machine-generated summary and the reference summaries using overlapping n-grams, word sequences that appear in both the machine-generated summary and the reference summaries. The most common n-grams used are unigrams, bigrams, and trigrams. ROUGE score calculates the recall of n-grams in the machine-generated summary by comparing them to the reference summaries.

**ROUGE-N:** ROUGE-N measures the overlap of n-grams (contiguous sequences of n words) between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the n-gram overlap. For example, ROUGE-1 (unigram) measures the overlap of single words, ROUGE-2 (bigram) measures the overlap of two-word sequences, and so on. ROUGE-N is often used to evaluate the grammatical correctness and fluency of generated text.

**ROUGE-L:** ROUGE-L measures the longest common subsequence (LCS) between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the length of the LCS. ROUGE-L is often used to evaluate the semantic similarity and content coverage of generated text, as it considers the common subsequence regardless of word order.

**ROUGE-S:** ROUGE-S measures the skip-bigram (bi-gram with at most one intervening word) overlap between the candidate text and the reference text. It computes the precision, recall, and F1-score based on the skip-bigram overlap. ROUGE-S is often used to evaluate the coherence and local cohesion of generated text, as it captures the semantic similarity between adjacent words.



In [ ]:
import numpy as np
bertscore_simple_averaged={}
bertscore_compressor_averaged={}
### your code ###
bertscore_simple = bertscore.compute(predictions=simple_answers, references=answers_merged, lang="en")["precision"]
bertscore_compressor = bertscore.compute(predictions=compressor_answers, references=answers_merged, lang="en")["precision"]
bertscore_simple_averaged = np.mean(bertscore_simple)
bertscore_compressor_averaged = np.mean(bertscore_compressor)
### your code ###
print("Simple system:")
print(bertscore_simple_averaged)
print("Compressor:")
print(bertscore_compressor_averaged)

Simple system:
0.8447270055611928
Compressor:
0.8374952793121337


Which model works better?

**Answer:**
The bertscore evaluator produces a float value indicating similarity for each answer-reference pair. Averaging them yields a value describing the quality of the answer. Judging by that rating, it seems as if the simple version performs better than the compressed version (simple: 0.845 > compressed: 0.837). Yet, one has to keep in mind that the compressed text is much much shorter than the simple text.
Therefore, much less context must be given and yet both versions perform similarly. Hence, incorporating the extreme conciseness of the context the compressed version "works better"?

#### ${\color{red}{Comments\ 2.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$